# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
file = "../../Instructions/output_data/cities.csv"
df = pd.read_csv(file)
df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
gmaps.configure(api_key=g_key)


In [20]:
locations = df[["Lat", "Lng"]]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=df['Humidity'],dissipating=False,max_intensity=100,point_radius=3)


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
weather_df = df.loc[(df['Max Temp']>65)  & (df['Max Temp']< 69) & (df['Wind Speed']<10) & (df['Cloudiness'] < 20)]
weather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93
30,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24
168,168,straumen,0,NO,1558378786,52,63.87,11.30,68.00,9.17
214,214,mount isa,0,AU,1558378794,56,-20.73,139.49,68.00,6.93
231,231,mocuba,1,MZ,1558378804,90,-16.84,36.99,65.76,2.42
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78


In [24]:
hotel_df = weather_df.dropna()
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93
168,168,straumen,0,NO,1558378786,52,63.87,11.30,68.00,9.17
214,214,mount isa,0,AU,1558378794,56,-20.73,139.49,68.00,6.93
231,231,mocuba,1,MZ,1558378804,90,-16.84,36.99,65.76,2.42
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

<ipython-input-25-9f48863c70ef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93,
168,168,straumen,0,NO,1558378786,52,63.87,11.30,68.00,9.17,
214,214,mount isa,0,AU,1558378794,56,-20.73,139.49,68.00,6.93,
231,231,mocuba,1,MZ,1558378804,90,-16.84,36.99,65.76,2.42,
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78,


In [33]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

params = {
    'radius': 5000,
    'type':'lodging',
    'key': g_key
}

In [38]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in hotel_df.iterrows():

    lat = row['Lat']
    lng = row['Lng']
    
    parameters['location'] = f"{lat},{lng}"
    
    hotel_name = requests.get(base_url, params=params)


    hotel_name = hotel_name.json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_name[0]['name']
    
    except (KeyError, IndexError):
        print("City is missing")
        
hotel_df

City is missing
City is missing
City is missing
City is missing
City is missing


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93,
168,168,straumen,0,NO,1558378786,52,63.87,11.30,68.00,9.17,
214,214,mount isa,0,AU,1558378794,56,-20.73,139.49,68.00,6.93,
231,231,mocuba,1,MZ,1558378804,90,-16.84,36.99,65.76,2.42,
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78,


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# Add marker layer ontop of heat map
coordinates = hotel_df[["Lat","Lng"]]
hotel_layer = gmaps.marker_layer(coordinates, info_box_content = hotel_info)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [41]:
fig

Figure(layout=FigureLayout(height='420px'))